#install

In [1]:
%%time 

# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
!pip install Shapely

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 40 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (2,388 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../python3-numpy_

In [2]:
from shapely.geometry import Point,Polygon
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt 
import folium
from folium import plugins
from folium.plugins import HeatMap
import plotly_express as px
from google.colab import drive
from IPython.display import display
drive.mount('/content/drive')

Mounted at /content/drive


#read

In [11]:
district_NE_path = "/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/District_NE.geojson"
gps_log_2019_4_1_09_path = "/content/drive/My Drive/gps_copy/2019-04/2019-04-01_09.zip"

In [10]:
!ls "/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data"

 District_NE.geojson  'Road Data'	        roadDB_NE_3digit.csv
 Open_Street_Map       roadDB_NE_1_2digit.csv


In [12]:
district_NE = gpd.read_file(district_NE_path)
gps = pd.read_csv(gps_log_2019_4_1_09_path, compression='zip')

In [13]:
gps

,time_stamp,unit_id,lat,lon,speed,unit_type
0,2019-04-01 09:00:00,0390002000000000000EE107666,13.542236,100.726311,28,7.0
1,2019-04-01 09:00:09,066000200000000003081840892,14.410528,99.902084,0,7.0
2,2019-04-01 09:00:16,0390002000000000000EE076202,16.955230,98.569691,22,7.0
3,2019-04-01 09:00:59,066000200000000005081858425,15.193460,101.608971,0,NaN
4,2019-04-01 09:00:05,0390002000000000000EE080921,13.546059,100.989644,23,7.0
...,...,...,...,...,...,...
4793086,2019-04-01 11:59:59,02800040000000GBN6104000952,12.370810,99.886230,0,8.0
4793087,2019-04-01 11:59:59,031000108966042115908951213,12.912299,101.479759,57,7.0
4793088,2019-04-01 11:59:59,033000700000864507034942051,13.812660,100.701248,51,NaN
4793089,2019-04-01 11:59:59,033000400000863835029881245,13.031306,101.366745,34,8.0


In [14]:
gps.unit_type.unique()

array([ 7., nan,  8.,  5.,  6.,  3.,  1.,  9.,  4.])

In [15]:
gps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4793091 entries, 0 to 4793090
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   time_stamp  object 
 1   unit_id     object 
 2   lat         float64
 3   lon         float64
 4   speed       int64  
 5   unit_type   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 219.4+ MB


In [21]:
gps.unit_type.value_counts().sum()

4330515

In [18]:
gps.unit_type.isnull().value_counts()

False    4330515
True      462576
Name: unit_type, dtype: int64

In [22]:
gps.unit_type.value_counts()

7.0    1506977
8.0    1285738
3.0     836571
1.0     303468
6.0     280790
5.0      67327
9.0      47640
4.0       2004
Name: unit_type, dtype: int64

#Clean

In [23]:
gps = gps[gps['unit_type'] != 1.0]

In [24]:
gps

,time_stamp,unit_id,lat,lon,speed,unit_type
0,2019-04-01 09:00:00,0390002000000000000EE107666,13.542236,100.726311,28,7.0
1,2019-04-01 09:00:09,066000200000000003081840892,14.410528,99.902084,0,7.0
2,2019-04-01 09:00:16,0390002000000000000EE076202,16.955230,98.569691,22,7.0
3,2019-04-01 09:00:59,066000200000000005081858425,15.193460,101.608971,0,NaN
4,2019-04-01 09:00:05,0390002000000000000EE080921,13.546059,100.989644,23,7.0
...,...,...,...,...,...,...
4793086,2019-04-01 11:59:59,02800040000000GBN6104000952,12.370810,99.886230,0,8.0
4793087,2019-04-01 11:59:59,031000108966042115908951213,12.912299,101.479759,57,7.0
4793088,2019-04-01 11:59:59,033000700000864507034942051,13.812660,100.701248,51,NaN
4793089,2019-04-01 11:59:59,033000400000863835029881245,13.031306,101.366745,34,8.0


In [25]:
gps = gps.dropna()

In [26]:
gps

,time_stamp,unit_id,lat,lon,speed,unit_type
0,2019-04-01 09:00:00,0390002000000000000EE107666,13.542236,100.726311,28,7.0
1,2019-04-01 09:00:09,066000200000000003081840892,14.410528,99.902084,0,7.0
2,2019-04-01 09:00:16,0390002000000000000EE076202,16.955230,98.569691,22,7.0
4,2019-04-01 09:00:05,0390002000000000000EE080921,13.546059,100.989644,23,7.0
5,2019-04-01 09:00:24,0110003GPSASIA-190300009184,16.239498,99.913248,0,8.0
...,...,...,...,...,...,...
4793085,2019-04-01 11:59:59,02800040000000GBNF620100022,14.567437,100.452537,63,7.0
4793086,2019-04-01 11:59:59,02800040000000GBN6104000952,12.370810,99.886230,0,8.0
4793087,2019-04-01 11:59:59,031000108966042115908951213,12.912299,101.479759,57,7.0
4793089,2019-04-01 11:59:59,033000400000863835029881245,13.031306,101.366745,34,8.0


#Algorithm

In [30]:
def check_district(lon,lat):
  point = Point(lon,lat)
  for index in district_NE.index:
    return point.within(district_NE.loc[index,'geometry'])

#Filter NE 

In [39]:
msk_NE = []
for index, row in gps.iterrows():
  if check_district(row.lon,row.lat):
    msk_NE.append(True)
  else:
    msk_NE.append(False)

In [41]:
len(msk_NE)

4027047

In [42]:
gps = gps[msk_NE]

In [43]:
gps

,time_stamp,unit_id,lat,lon,speed,unit_type
601,2019-04-01 09:00:41,060000300000863835026178876,14.993836,102.168104,0,7.0
641,2019-04-01 09:00:08,036000200000864507032202813,15.089150,102.111580,0,7.0
1060,2019-04-01 09:00:45,030000400000000ESP_006_2495,14.899563,102.068928,0,8.0
1399,2019-04-01 09:00:21,030000400000000ESP_006_2532,15.012812,102.124180,0,7.0
1579,2019-04-01 09:00:50,030000400000000ESP_006_2156,15.028908,102.139530,0,7.0
...,...,...,...,...,...,...
4792301,2019-04-01 11:59:50,001001200000011145100000006,14.920464,102.166565,0,9.0
4792400,2019-04-01 11:59:51,077000500000000000000108983,14.914882,102.150285,0,7.0
4792958,2019-04-01 11:59:58,02800040000000GBNF611200247,14.935387,101.990776,0,8.0
4793038,2019-04-01 11:59:59,001000800000000178030113731,14.915882,102.160300,0,7.0


In [45]:
len(gps.unit_id.unique())

2286

#plot

In [46]:
lat = gps.lat
lon =gps.lon
coord = pd.DataFrame()
coord.insert(0,"lat",lat)
coord.insert(1,"lon",lon)
coord.reset_index(inplace=True)
heat_data = [[row['lat'],row['lon']] for index, row in coord.iterrows()]

In [47]:
mapit = folium.Map( location=[ 17.87914890447798,102.7390305957055], zoom_start=6 )
HeatMap(heat_data).add_to(mapit)
display(mapit)